In [2]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import math
import json
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from IPython.display import Image
import seaborn as sns
# import missingno as msnco

In [3]:
import warnings

# 경고 메시지를 무시하고 숨기거나
warnings.filterwarnings(action='ignore')

In [4]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
sample_df = pd.read_csv('sample_submission.csv')

In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 19 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   ID                        1000000 non-null  object 
 1   Month                     1000000 non-null  int64  
 2   Day_of_Month              1000000 non-null  int64  
 3   Estimated_Departure_Time  890981 non-null   float64
 4   Estimated_Arrival_Time    890960 non-null   float64
 5   Cancelled                 1000000 non-null  int64  
 6   Diverted                  1000000 non-null  int64  
 7   Origin_Airport            1000000 non-null  object 
 8   Origin_Airport_ID         1000000 non-null  int64  
 9   Origin_State              890985 non-null   object 
 10  Destination_Airport       1000000 non-null  object 
 11  Destination_Airport_ID    1000000 non-null  int64  
 12  Destination_State         890921 non-null   object 
 13  Distance                  10

In [6]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 18 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   ID                        1000000 non-null  object 
 1   Month                     1000000 non-null  int64  
 2   Day_of_Month              1000000 non-null  int64  
 3   Estimated_Departure_Time  891016 non-null   float64
 4   Estimated_Arrival_Time    890952 non-null   float64
 5   Cancelled                 1000000 non-null  int64  
 6   Diverted                  1000000 non-null  int64  
 7   Origin_Airport            1000000 non-null  object 
 8   Origin_Airport_ID         1000000 non-null  int64  
 9   Origin_State              893495 non-null   object 
 10  Destination_Airport       1000000 non-null  object 
 11  Destination_Airport_ID    1000000 non-null  int64  
 12  Destination_State         893477 non-null   object 
 13  Distance                  10

In [9]:
train_df.isna().sum()

ID                               0
Month                            0
Day_of_Month                     0
Estimated_Departure_Time    109019
Estimated_Arrival_Time      109040
Cancelled                        0
Diverted                         0
Origin_Airport                   0
Origin_Airport_ID                0
Origin_State                109015
Destination_Airport              0
Destination_Airport_ID           0
Destination_State           109079
Distance                         0
Airline                     108920
Carrier_Code(IATA)          108990
Carrier_ID(DOT)             108997
Tail_Number                      0
Delay                       744999
dtype: int64

In [10]:
test_df.isna().sum()

ID                               0
Month                            0
Day_of_Month                     0
Estimated_Departure_Time    108984
Estimated_Arrival_Time      109048
Cancelled                        0
Diverted                         0
Origin_Airport                   0
Origin_Airport_ID                0
Origin_State                106505
Destination_Airport              0
Destination_Airport_ID           0
Destination_State           106523
Distance                         0
Airline                     106527
Carrier_Code(IATA)          108993
Carrier_ID(DOT)             109006
Tail_Number                      0
dtype: int64

# airport 별 시간 

In [14]:
#도착시간이 출발시간보다 큰 경우만
df = train_df[train_df['Estimated_Arrival_Time']>train_df['Estimated_Departure_Time']]
df

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay
1,TRAIN_000001,8,15,740.0,1024.0,0,0,ORD,13930,Illinois,SLC,14869,Utah,1250.0,SkyWest Airlines Inc.,UA,20304.0,N125SY,NaN
2,TRAIN_000002,9,6,1610.0,1805.0,0,0,CLT,11057,North Carolina,LGA,12953,New York,544.0,American Airlines Inc.,AA,19805.0,N103US,NaN
3,TRAIN_000003,7,10,905.0,1735.0,0,0,LAX,12892,California,EWR,11618,New Jersey,2454.0,United Air Lines Inc.,UA,NaN,N595UA,NaN
4,TRAIN_000004,1,11,900.0,1019.0,0,0,SFO,14771,California,ACV,10157,California,250.0,SkyWest Airlines Inc.,UA,20304.0,N161SY,NaN
6,TRAIN_000006,1,20,1742.0,1903.0,0,0,EWR,11618,New Jersey,BOS,10721,Massachusetts,200.0,United Air Lines Inc.,UA,NaN,N66825,Not_Delayed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,TRAIN_999995,9,18,936.0,1243.0,0,0,ORD,13930,NaN,PHL,14100,NaN,678.0,United Air Lines Inc.,UA,19977.0,N477UA,NaN
999996,TRAIN_999996,5,30,920.0,1028.0,0,0,FAR,11637,NaN,MSP,13487,Minnesota,223.0,SkyWest Airlines Inc.,DL,NaN,N439SW,NaN
999997,TRAIN_999997,6,28,800.0,1340.0,0,0,OAK,13796,NaN,HOU,12191,Texas,1642.0,Southwest Airlines Co.,WN,19393.0,N230WN,NaN
999998,TRAIN_999998,9,27,1613.0,1824.0,0,0,BNA,10693,Tennessee,ATL,10397,NaN,214.0,Delta Air Lines Inc.,DL,19790.0,N968DL,NaN


In [15]:
# #시간차이와 거리평균 딕셔너리 형태로 저장
# time_diff_dict = {}
# distance_mean_dict = {}
# def time_diff(train_df, state):
#     OA = train_df[train_df['Origin_State']==state]
#     destination_list = OA.Destination_State.unique().tolist()
# #     time_diff_dict = {}
# #     distance_mean_dict = {}
#     for d in destination_list:
#         st_oa = OA[OA['Destination_State']==d]
#         time_diff = (st_oa['Estimated_Arrival_Time'] - st_oa['Estimated_Departure_Time']).mean()
#         time_diff_dict[state,d] = (time_diff)
#         distance_mean_dict[state,d] = (st_oa['Distance'].mean())
    
#     return time_diff_dict, distance_mean_dict

In [32]:
#시간차이 딕셔너리 형태로 저장
time_diff_dict = {}

def time_diff(train_df, airport):
    OA = train_df[train_df['Origin_Airport']==airport]
    destination_airport_list = OA.Destination_Airport.unique().tolist()
    
    for d in destination_airport_list:
        air_oa = OA[OA['Destination_Airport']==d]
        time_diff = (air_oa['Estimated_Arrival_Time'] - air_oa['Estimated_Departure_Time']).mean()
        time_diff_dict[airport, d] = round(time_diff)
    
    return time_diff_dict

In [33]:
airport_list = df['Origin_Airport'].unique().tolist()

In [34]:
for i in airport_list:
    time_diff(df, i)

In [35]:
len(time_diff_dict)

6607

In [36]:
time_diff_dict

{('ORD', 'SLC'): 264,
 ('ORD', 'MCI'): 152,
 ('ORD', 'LEX'): 234,
 ('ORD', 'STL'): 131,
 ('ORD', 'CHA'): 270,
 ('ORD', 'CMH'): 235,
 ('ORD', 'CRW'): 260,
 ('ORD', 'BUF'): 257,
 ('ORD', 'AUS'): 277,
 ('ORD', 'IND'): 211,
 ('ORD', 'UIN'): 117,
 ('ORD', 'MLI'): 100,
 ('ORD', 'PHL'): 301,
 ('ORD', 'DAY'): 223,
 ('ORD', 'MSP'): 153,
 ('ORD', 'IAD'): 277,
 ('ORD', 'LGA'): 316,
 ('ORD', 'FLL'): 407,
 ('ORD', 'CLE'): 230,
 ('ORD', 'SCE'): 273,
 ('ORD', 'IAH'): 273,
 ('ORD', 'SDF'): 245,
 ('ORD', 'MDT'): 274,
 ('ORD', 'TOL'): 196,
 ('ORD', 'FSD'): 180,
 ('ORD', 'MEM'): 187,
 ('ORD', 'EYW'): 417,
 ('ORD', 'SGF'): 177,
 ('ORD', 'PWM'): 331,
 ('ORD', 'PIA'): 98,
 ('ORD', 'MSY'): 236,
 ('ORD', 'SAN'): 240,
 ('ORD', 'DCA'): 277,
 ('ORD', 'EAU'): 137,
 ('ORD', 'CLT'): 303,
 ('ORD', 'MCO'): 364,
 ('ORD', 'LAS'): 197,
 ('ORD', 'CHO'): 287,
 ('ORD', 'CMX'): 229,
 ('ORD', 'COU'): 144,
 ('ORD', 'XNA'): 189,
 ('ORD', 'CHS'): 328,
 ('ORD', 'ROC'): 262,
 ('ORD', 'ABE'): 297,
 ('ORD', 'FWA'): 206,
 ('ORD', 'M

## time_diff 컬럼추가

In [31]:
train_df

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay,time_diff
0,TRAIN_000000,4,15,NaN,NaN,0,0,OKC,13851,Oklahoma,HOU,12191,Texas,419.0,Southwest Airlines Co.,WN,19393.0,N7858A,NaN,147.576687
1,TRAIN_000001,8,15,740.0,1024.0,0,0,ORD,13930,Illinois,SLC,14869,Utah,1250.0,SkyWest Airlines Inc.,UA,20304.0,N125SY,NaN,263.664723
2,TRAIN_000002,9,6,1610.0,1805.0,0,0,CLT,11057,North Carolina,LGA,12953,New York,544.0,American Airlines Inc.,AA,19805.0,N103US,NaN,195.689024
3,TRAIN_000003,7,10,905.0,1735.0,0,0,LAX,12892,California,EWR,11618,New Jersey,2454.0,United Air Lines Inc.,UA,NaN,N595UA,NaN,823.909707
4,TRAIN_000004,1,11,900.0,1019.0,0,0,SFO,14771,California,ACV,10157,California,250.0,SkyWest Airlines Inc.,UA,20304.0,N161SY,NaN,126.664234
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,TRAIN_999995,9,18,936.0,1243.0,0,0,ORD,13930,NaN,PHL,14100,NaN,678.0,United Air Lines Inc.,UA,19977.0,N477UA,NaN,300.917031
999996,TRAIN_999996,5,30,920.0,1028.0,0,0,FAR,11637,NaN,MSP,13487,Minnesota,223.0,SkyWest Airlines Inc.,DL,NaN,N439SW,NaN,115.588785
999997,TRAIN_999997,6,28,800.0,1340.0,0,0,OAK,13796,NaN,HOU,12191,Texas,1642.0,Southwest Airlines Co.,WN,19393.0,N230WN,NaN,572.643678
999998,TRAIN_999998,9,27,1613.0,1824.0,0,0,BNA,10693,Tennessee,ATL,10397,NaN,214.0,Delta Air Lines Inc.,DL,19790.0,N968DL,NaN,217.780488


In [37]:
train_df['time_diff']=np.nan

In [38]:
#train_df에 시간차이의 평균과 거리 평균 컬럼으로 추가
for i in range(len(train_df)):
    if (train_df['Origin_Airport'].iloc[i], train_df['Destination_Airport'].iloc[i]) in time_diff_dict.keys():
        train_df['time_diff'].iloc[i] = time_diff_dict[train_df['Origin_Airport'].iloc[i], train_df['Destination_Airport'].iloc[i]]

In [39]:
train_df

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay,time_diff
0,TRAIN_000000,4,15,NaN,NaN,0,0,OKC,13851,Oklahoma,HOU,12191,Texas,419.0,Southwest Airlines Co.,WN,19393.0,N7858A,NaN,148.0
1,TRAIN_000001,8,15,740.0,1024.0,0,0,ORD,13930,Illinois,SLC,14869,Utah,1250.0,SkyWest Airlines Inc.,UA,20304.0,N125SY,NaN,264.0
2,TRAIN_000002,9,6,1610.0,1805.0,0,0,CLT,11057,North Carolina,LGA,12953,New York,544.0,American Airlines Inc.,AA,19805.0,N103US,NaN,196.0
3,TRAIN_000003,7,10,905.0,1735.0,0,0,LAX,12892,California,EWR,11618,New Jersey,2454.0,United Air Lines Inc.,UA,NaN,N595UA,NaN,824.0
4,TRAIN_000004,1,11,900.0,1019.0,0,0,SFO,14771,California,ACV,10157,California,250.0,SkyWest Airlines Inc.,UA,20304.0,N161SY,NaN,127.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,TRAIN_999995,9,18,936.0,1243.0,0,0,ORD,13930,NaN,PHL,14100,NaN,678.0,United Air Lines Inc.,UA,19977.0,N477UA,NaN,301.0
999996,TRAIN_999996,5,30,920.0,1028.0,0,0,FAR,11637,NaN,MSP,13487,Minnesota,223.0,SkyWest Airlines Inc.,DL,NaN,N439SW,NaN,116.0
999997,TRAIN_999997,6,28,800.0,1340.0,0,0,OAK,13796,NaN,HOU,12191,Texas,1642.0,Southwest Airlines Co.,WN,19393.0,N230WN,NaN,573.0
999998,TRAIN_999998,9,27,1613.0,1824.0,0,0,BNA,10693,Tennessee,ATL,10397,NaN,214.0,Delta Air Lines Inc.,DL,19790.0,N968DL,NaN,218.0


## 출발,도착예정시간 결측치 채우기 

In [40]:
for i in range(len(train_df)):
    #출발시간은 데이터가 존재하고, 도착시간은 존재하지않을 때
    if train_df['Estimated_Departure_Time'].iloc[i].astype(str)!='nan' and train_df['Estimated_Arrival_Time'].iloc[i].astype(str)=='nan': 
        train_df['Estimated_Arrival_Time'].iloc[i] = train_df['Estimated_Departure_Time'].iloc[i] + train_df['time_diff'].iloc[i]
     #출발시간은 존재하지 않고, 도착시간은 존재할때    
    elif train_df['Estimated_Departure_Time'].iloc[i].astype(str)=='nan' and train_df['Estimated_Arrival_Time'].iloc[i].astype(str)!='nan': 
        train_df['Estimated_Departure_Time'].iloc[i] = train_df['Estimated_Arrival_Time'].iloc[i] - train_df['time_diff'].iloc[i]


In [41]:
train_df

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay,time_diff
0,TRAIN_000000,4,15,NaN,NaN,0,0,OKC,13851,Oklahoma,HOU,12191,Texas,419.0,Southwest Airlines Co.,WN,19393.0,N7858A,NaN,148.0
1,TRAIN_000001,8,15,740.0,1024.0,0,0,ORD,13930,Illinois,SLC,14869,Utah,1250.0,SkyWest Airlines Inc.,UA,20304.0,N125SY,NaN,264.0
2,TRAIN_000002,9,6,1610.0,1805.0,0,0,CLT,11057,North Carolina,LGA,12953,New York,544.0,American Airlines Inc.,AA,19805.0,N103US,NaN,196.0
3,TRAIN_000003,7,10,905.0,1735.0,0,0,LAX,12892,California,EWR,11618,New Jersey,2454.0,United Air Lines Inc.,UA,NaN,N595UA,NaN,824.0
4,TRAIN_000004,1,11,900.0,1019.0,0,0,SFO,14771,California,ACV,10157,California,250.0,SkyWest Airlines Inc.,UA,20304.0,N161SY,NaN,127.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,TRAIN_999995,9,18,936.0,1243.0,0,0,ORD,13930,NaN,PHL,14100,NaN,678.0,United Air Lines Inc.,UA,19977.0,N477UA,NaN,301.0
999996,TRAIN_999996,5,30,920.0,1028.0,0,0,FAR,11637,NaN,MSP,13487,Minnesota,223.0,SkyWest Airlines Inc.,DL,NaN,N439SW,NaN,116.0
999997,TRAIN_999997,6,28,800.0,1340.0,0,0,OAK,13796,NaN,HOU,12191,Texas,1642.0,Southwest Airlines Co.,WN,19393.0,N230WN,NaN,573.0
999998,TRAIN_999998,9,27,1613.0,1824.0,0,0,BNA,10693,Tennessee,ATL,10397,NaN,214.0,Delta Air Lines Inc.,DL,19790.0,N968DL,NaN,218.0


In [42]:
train_df.isna().sum()

ID                               0
Month                            0
Day_of_Month                     0
Estimated_Departure_Time     11943
Estimated_Arrival_Time       11930
Cancelled                        0
Diverted                         0
Origin_Airport                   0
Origin_Airport_ID                0
Origin_State                109015
Destination_Airport              0
Destination_Airport_ID           0
Destination_State           109079
Distance                         0
Airline                     108920
Carrier_Code(IATA)          108990
Carrier_ID(DOT)             108997
Tail_Number                      0
Delay                       744999
time_diff                     2392
dtype: int64

In [54]:
#출발 시간이 음수인 경우 2400 더해주기 
train_df['Estimated_Departure_Time'] = train_df['Estimated_Departure_Time'].apply(lambda x : x + 2400 if x<0 else x)
#도착 시간이 2400이 넘는 경우 2400 빼주기
train_df['Estimated_Arrival_Time'] = train_df['Estimated_Arrival_Time'].apply(lambda x : x -2400 if x >= 2400 else x)

In [55]:
train_df

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay,time_diff
0,TRAIN_000000,4,15,NaN,NaN,0,0,OKC,13851,Oklahoma,HOU,12191,Texas,419.0,Southwest Airlines Co.,WN,19393.0,N7858A,NaN,148.0
1,TRAIN_000001,8,15,740.0,1024.0,0,0,ORD,13930,Illinois,SLC,14869,Utah,1250.0,SkyWest Airlines Inc.,UA,20304.0,N125SY,NaN,264.0
2,TRAIN_000002,9,6,1610.0,1805.0,0,0,CLT,11057,North Carolina,LGA,12953,New York,544.0,American Airlines Inc.,AA,19805.0,N103US,NaN,196.0
3,TRAIN_000003,7,10,905.0,1735.0,0,0,LAX,12892,California,EWR,11618,New Jersey,2454.0,United Air Lines Inc.,UA,NaN,N595UA,NaN,824.0
4,TRAIN_000004,1,11,900.0,1019.0,0,0,SFO,14771,California,ACV,10157,California,250.0,SkyWest Airlines Inc.,UA,20304.0,N161SY,NaN,127.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,TRAIN_999995,9,18,936.0,1243.0,0,0,ORD,13930,NaN,PHL,14100,NaN,678.0,United Air Lines Inc.,UA,19977.0,N477UA,NaN,301.0
999996,TRAIN_999996,5,30,920.0,1028.0,0,0,FAR,11637,NaN,MSP,13487,Minnesota,223.0,SkyWest Airlines Inc.,DL,NaN,N439SW,NaN,116.0
999997,TRAIN_999997,6,28,800.0,1340.0,0,0,OAK,13796,NaN,HOU,12191,Texas,1642.0,Southwest Airlines Co.,WN,19393.0,N230WN,NaN,573.0
999998,TRAIN_999998,9,27,1613.0,1824.0,0,0,BNA,10693,Tennessee,ATL,10397,NaN,214.0,Delta Air Lines Inc.,DL,19790.0,N968DL,NaN,218.0


# Origin_state, Destination_State 결측치 채우기
출발공항, 도착 공항은 결측치가 존재하지 않음.
출발 공항이 같다면 출발 state가 동일하고 도착 공항이 같다면 도착 state가 동일 

In [170]:
#출발 공항이 같을 경우 같은 출발 state
train_df['Origin_State'] = train_df.groupby('Origin_Airport')['Origin_State'].ffill().bfill()
#도착 공항이 같은 경우 같은 도착 state
train_df['Destination_State'] = train_df.groupby('Destination_Airport')['Destination_State'].ffill().bfill()

In [175]:
train_df.isna().sum()

ID                               0
Month                            0
Day_of_Month                     0
Estimated_Departure_Time     11943
Estimated_Arrival_Time       11930
Cancelled                        0
Diverted                         0
Origin_Airport                   0
Origin_Airport_ID                0
Origin_State                     0
Destination_Airport              0
Destination_Airport_ID           0
Destination_State                0
Distance                         0
Airline                     108920
Carrier_Code(IATA)          108990
Carrier_ID(DOT)             108997
Tail_Number                      0
Delay                       744999
time_diff                     2392
dtype: int64

순서 : airline > carrier_id > carrier_code > (시간은 채울지 말지) > delay  순으로 self_training 

In [ ]:
import numpy as np
import random
import os
import gc

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) # Seed 고정

In [ ]:
import matplotlib
#train 데이터 100만개에서 각 ID별 결측치 개수
column = ['결손값 개수']
axis_1 = pd.DataFrame(data = train.isna().sum(axis=1), columns = column)

#결측치 많은 데이터 순으로 정렬
axis_1 = axis_1.sort_values(by = '결손값 개수', ascending = False) 

# 컬럼 별 결측치 개수 많은 순으로 정렬/ 8개의 컬럼 결측치 존재
train.isnull().sum(axis=0)
train_row_NaN = pd.DataFrame(train.isnull().sum(axis=0), columns = column)
train_row_NaN = train_row_NaN.sort_values(by = '결손값 개수', ascending = False)

#결손값 비율 
train_row_su = train.shape[0]
train_row_NaN['결손값_ratio'] = train_row_NaN['결손값 개수']/train_row_su 



In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

plt.figure(figsize = (16,8))
x = np.arange(len(train_row_NaN.index))
plt.subplot(1, 2, 1)
plt.xticks(x, train_row_NaN.index)
plt.xticks(rotation=90)
plt.legend(handles=[mpatches.Patch(color='orange'),
                    mpatches.Patch(color='red')], 
           labels=['Partial values in columns are NaN', 'All values in columns are NaN'])

def color(data):
    if data == 1:
        return 'red'
    else:
        return 'orange'
plt.bar(x, train_row_NaN['결손값_ratio'], color = [color(i) for i in train_row_NaN['결손값_ratio']])
plt.title('Train data NaN check')

test.isnull().sum(axis=0)
test_row_NaN = pd.DataFrame(test.isnull().sum(axis=0), columns = column)
test_row_NaN = test_row_NaN.sort_values(by = '결손값 개수', ascending = False)
test_row_su = test.shape[0]
test_row_NaN['결손값_ratio'] = test_row_NaN['결손값 개수']/test_row_su

plt.subplot(1, 2, 2)
x2 = np.arange(len(test_row_NaN.index))
plt.xticks(x2, test_row_NaN.index)
plt.xticks(rotation=90)
plt.legend(handles=[mpatches.Patch(color='orange'),
                    mpatches.Patch(color='red')], 
           labels=['Partial values in columns are NaN', 'All values in columns are NaN'])
plt.bar(x2, test_row_NaN['결손값_ratio'], color = [color(i) for i in test_row_NaN['결손값_ratio']])
plt.title('Test data NaN check')

print(train_row_NaN)
print(test_row_NaN)
plt.show()

In [ ]:
# 준지도 학습 전 라벨링
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

In [ ]:
train_lb = train_df.__deepcopy__() #복사
test_lb = test_df.__deepcopy__() #복사 

label_st = train_lb.drop(columns = ['Airline']) #airline 컬럼 삭제

#airline이 결측치가 아니였던 데이터 / 891080 row x 18col
train_st = train_lb[(train_lb['Airline'].astype(str) != 'nan')].drop(columns = ['Airline']) 

#airline이 결측치였던 데이터/ 108920 row x 18 col
X_nan = train_lb[(train_lb['Airline'].astype(str) == 'nan')].drop(columns = ['Airline']) #108920

#test 데이터에서 airline이 결측치였던 데이터.  106527 row x 17 col
test_nan = test_lb[(test_lb['Airline'].astype(str) == 'None')|(test_lb['Airline'].astype(str) == 'nan')].drop(columns = ['Airline'])

# NULL이 없으면서 라벨링이 필요한 열들 정리
qual_col = ['Origin_Airport', 'Origin_State', 'Destination_Airport', 'Destination_State', 'Tail_Number']

for i in qual_col:
    le = LabelEncoder()
    le=le.fit(label_st[i])
    for label in np.unique(test[i].dropna()):
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    train_lb[i] = le.transform(train_lb[i])
    train_st[i]=le.transform(train_st[i])
    X_nan[i]=le.transform(X_nan[i])
    test_nan[i]=le.transform(test_nan[i])

In [ ]:
# Airline self_training
original_col = ['Cancelled','Diverted','Origin_Airport','Origin_Airport_ID', 'Origin_State', 'Destination_Airport', 'Destination_Airport_ID', 'Destination_State', 'Tail_Number', 'Month', 'Day_of_Month','Distance']

X = train_st[original_col]
y = train_lb[(train_lb['Airline'].astype(str) != 'nan')][original_col + ['Airline']]['Airline']

stclf = SelfTrainingClassifier(
    base_estimator = RandomForestClassifier(n_estimators = 100),
    verbose = True)

stclf.fit(X, y)   

In [ ]:
# 결과값 삽입

X_nan['Airline_pred'] = stclf.predict(X_nan[original_col])
test_nan['Airline_pred'] = stclf.predict(test_nan[original_col])
train_lb.loc[train_lb['Airline'].astype(str) == 'nan', 'Airline'] = X_nan['Airline_pred']
test_lb.loc[test_lb['Airline'].astype(str) == 'nan', 'Airline'] = test_nan['Airline_pred']

In [ ]:
#airline 학습 후 결과 그래프로 
import matplotlib

matplotlib.rcParams['font.family'] = 'NanumBarunGothic'
matplotlib.rcParams['axes.unicode_minus'] = False

column = ['결손값 개수']
axis_1 = pd.DataFrame(data = train.isnull().sum(axis=1), columns = column)
axis_1 = axis_1.sort_values(by = '결손값 개수', ascending = False)

train_lb.isnull().sum(axis=0)
train_row_NaN = pd.DataFrame(train_lb.isnull().sum(axis=0), columns = column)
train_row_NaN = train_row_NaN.sort_values(by = '결손값 개수', ascending = False)
train_row_su = train.shape[0]
train_row_NaN['결손값_ratio'] = train_row_NaN['결손값 개수']/train_row_su

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

plt.figure(figsize = (16,8))
x = np.arange(len(train_row_NaN.index))
plt.subplot(1, 2, 1)
plt.xticks(x, train_row_NaN.index)
plt.xticks(rotation=90)
plt.legend(handles=[mpatches.Patch(color='orange'),
                    mpatches.Patch(color='red')], 
           labels=['Partial values in columns are NaN', 'All values in columns are NaN'])

def color(data):
    if data == 1:
        return 'red'
    else:
        return 'orange'
plt.bar(x, train_row_NaN['결손값_ratio'], color = [color(i) for i in train_row_NaN['결손값_ratio']])
plt.title('After Self_training Train data NaN check')

test_lb.isnull().sum(axis=0)
test_row_NaN = pd.DataFrame(test_lb.isnull().sum(axis=0), columns = column)
test_row_NaN = test_row_NaN.sort_values(by = '결손값 개수', ascending = False)
test_row_su = test.shape[0]
test_row_NaN['결손값_ratio'] = test_row_NaN['결손값 개수']/test_row_su

plt.subplot(1, 2, 2)
x2 = np.arange(len(test_row_NaN.index))
plt.xticks(x2, test_row_NaN.index)
plt.xticks(rotation=90)
plt.legend(handles=[mpatches.Patch(color='orange'),
                    mpatches.Patch(color='red')], 
           labels=['Partial values in columns are NaN', 'All values in columns are NaN'])
plt.bar(x2, test_row_NaN['결손값_ratio'], color = [color(i) for i in test_row_NaN['결손값_ratio']])
plt.title('After Self_training Test data NaN check')

print(train_row_NaN)
print(test_row_NaN)
plt.show()

'Carrier_ID(DOT)' 

In [ ]:
# 준지도 학습 전 라벨링
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

label_st = train_lb.drop(columns = ['Carrier_ID(DOT)'])
train_st = train_lb[(train_lb['Carrier_ID(DOT)'].astype(str) != 'nan')].drop(columns = ['Carrier_ID(DOT)'])
X_nan = train_lb[(train_lb['Carrier_ID(DOT)'].astype(str) == 'nan')].drop(columns = ['Carrier_ID(DOT)'])
test_nan = test_lb[(test_lb['Carrier_ID(DOT)'].astype(str) == 'None')|(test_lb['Carrier_ID(DOT)'].astype(str) == 'nan')].drop(columns = ['Carrier_ID(DOT)'])

#NULL이 없으면서 라벨링이 필요한 열들 정리
qual_col = ['Origin_Airport', 'Origin_State', 'Destination_Airport', 'Destination_State', 'Tail_Number']

for i in qual_col:
    le = LabelEncoder()
    le=le.fit(label_st[i])
    for label in np.unique(test[i].dropna()):
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    train_lb[i] = le.transform(train_lb[i])
    train_st[i]=le.transform(train_st[i])
    X_nan[i]=le.transform(X_nan[i])
    test_nan[i]=le.transform(test_nan[i])

In [ ]:
# carrier_id(dot) self_training
original_col = ['Cancelled','Diverted','Origin_Airport','Origin_Airport_ID', 'Origin_State', 'Destination_Airport', 'Destination_Airport_ID', 'Destination_State', 'Tail_Number', 'Month', 'Day_of_Month','Distance']

X = train_st[original_col]
y = train_lb[(train_lb['Carrier_ID(DOT)'].astype(str) != 'nan')][original_col + ['Carrier_ID(DOT)']]['Carrier_ID(DOT)']

stclf = SelfTrainingClassifier(
    base_estimator = RandomForestClassifier(n_estimators = 100),
    verbose = True)

stclf.fit(X, y)   

In [ ]:
# 결과값 삽입

X_nan['Carrier_ID(DOT)_pred'] = stclf.predict(X_nan[original_col])
test_nan['Carrier_ID(DOT)_pred'] = stclf.predict(test_nan[original_col])
# train_lb.loc[train_lb['Carrier_ID(DOT)'].astype(str) == 'nan', 'Carrier_ID(DOT)'] = X_nan['Carrier_ID(DOT)_pred']
# test_lb.loc[test_lb['Carrier_ID(DOT)'].astype(str) == 'nan', 'Carrier_ID(DOT)'] = test_nan['Carrier_ID(DOT)_pred']

'Carrier_Code(IATA)'

In [ ]:
# 준지도 학습 전 라벨링
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

label_st = train_lb.drop(columns = ['Carrier_ID(IATA)'])
train_st = train_lb[(train_lb['Carrier_ID(IATA)'].astype(str) != 'nan')].drop(columns = ['Carrier_ID(IATA)'])
X_nan = train_lb[(train_lb['Carrier_ID(IATA)'].astype(str) == 'nan')].drop(columns = ['Carrier_ID(IATA)'])
test_nan = test_lb[(test_lb['Carrier_ID(IATA)'].astype(str) == 'None')|(test_lb['Carrier_ID(IATA)'].astype(str) == 'nan')].drop(columns = ['Carrier_ID(IATA)'])

# NULL이 없으면서 라벨링이 필요한 열들 정리
qual_col = ['Origin_Airport', 'Origin_State', 'Destination_Airport', 'Destination_State', 'Tail_Number']

for i in qual_col:
    le = LabelEncoder()
    le=le.fit(label_st[i])
    for label in np.unique(test[i].dropna()):
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    train_lb[i] = le.transform(train_lb[i])
    train_st[i]=le.transform(train_st[i])
    X_nan[i]=le.transform(X_nan[i])
    test_nan[i]=le.transform(test_nan[i])

In [ ]:
#Carrier_ID(IATA) self-training
original_col = ['Cancelled','Diverted','Origin_Airport','Origin_Airport_ID', 'Origin_State', 'Destination_Airport', 'Destination_Airport_ID', 'Destination_State', 'Tail_Number', 'Month', 'Day_of_Month','Distance']

X = train_st[original_col]
y = train_lb[(train_lb['Carrier_ID(IATA)'].astype(str) != 'nan')][original_col + ['Carrier_ID(IATA)']]['Carrier_ID(IATA)']

stclf = SelfTrainingClassifier(
    base_estimator = RandomForestClassifier(n_estimators = 100),
    verbose = True)

stclf.fit(X, y)   

In [ ]:
# 결과값 삽입

X_nan['Carrier_ID(IATA)_pred'] = stclf.predict(X_nan[original_col])
test_nan['Carrier_ID(IATA)_pred'] = stclf.predict(test_nan[original_col])
train_lb.loc[train_lb['Carrier_ID(IATA)'].astype(str) == 'nan', 'Carrier_ID(IATA)'] = X_nan['Carrier_ID(IATA)_pred']
test_lb.loc[test_lb['Carrier_ID(IATA)'].astype(str) == 'nan', 'Carrier_ID(IATA)'] = test_nan['Carrier_ID(IATA)_pred']

'Delay' self_training 하기 전에 시간 결측치 삭제 or 시간 self_training

In [ ]:
# 준지도 학습 전 라벨링
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

label_st = train_lb.drop(columns = ['Delay'])
train_st = train_lb[(train_lb['Delay'].astype(str) != 'nan')].drop(columns = ['Delay'])
X_nan = train_lb[(train_lb['Delay'].astype(str) == 'nan')].drop(columns = ['Delay'])
test_nan = test_lb[(test_lb['Delay'].astype(str) == 'None')|(test_lb['Delay'].astype(str) == 'nan')].drop(columns = ['Delay'])

# NULL이 없으면서 라벨링이 필요한 열들 정리
qual_col = ['Origin_Airport', 'Origin_State', 'Destination_Airport', 'Destination_State', 'Tail_Number']

for i in qual_col:
    le = LabelEncoder()
    le=le.fit(label_st[i])
    for label in np.unique(test[i].dropna()):
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    train_lb[i] = le.transform(train_lb[i])
    train_st[i]=le.transform(train_st[i])
    X_nan[i]=le.transform(X_nan[i])
    test_nan[i]=le.transform(test_nan[i])

In [ ]:
#Delay self-training
original_col = ['Cancelled','Diverted','Origin_Airport','Origin_Airport_ID', 'Origin_State', 'Destination_Airport', 'Destination_Airport_ID', 'Destination_State', 'Tail_Number', 'Month', 'Day_of_Month','Distance']

X = train_st[original_col]
y = train_lb[(train_lb['Delay'].astype(str) != 'nan')][original_col + ['Delay']]['Delay']

stclf = SelfTrainingClassifier(
    base_estimator = RandomForestClassifier(n_estimators = 100),
    verbose = True)

stclf.fit(X, y)  

In [ ]:
# 결과값 삽입

X_nan['Delay_pred'] = stclf.predict(X_nan[original_col])
test_nan['Delay_pred'] = stclf.predict(test_nan[original_col])
train_lb.loc[train_lb['Delay'].astype(str) == 'nan', 'Delay'] = X_nan['Delay_pred']
test_lb.loc[test_lb['Delay'].astype(str) == 'nan', 'Delay'] = test_nan['Delay_pred']

delay까지 self_training완료 후 autogluon으로 ~ 